# Homework 2

# Set up

## Installing packages

In [1]:
!pip install requests PyPDF2 gdown
!pip install 'markitdown[pdf]'
!pip install langchain_mcp_adapters langchain_google_genai langchain-openai

## Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `VERTEX_API_KEY`.


1.   Look for the key icon on the left panel of your colab.
2.   Under `Name`, create `VERTEX_API_KEY`.
3. Copy your key to `Value`.

If you cannot use VERTEX_API_KEY, you can use deepseek models via `DEEPSEEK_API_KEY`. It does not affect your score.



In [2]:
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
# DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')

# Download sample CVs

## Downloading sample_cv.pdf
The codes below download the sample CV


In [3]:
import os
import gdown

folder_id = "1RsWfaco_wxDS70DVd7-sfzvwqN7_US_7"
folder_url = f"https://drive.google.com/drive/folders/{folder_id}"

output_dir = "downloaded_cvs"
os.makedirs(output_dir, exist_ok=True)

gdown.download_folder(
    url=folder_url,
    output=output_dir,
    quiet=False,
    use_cookies=False
)

Retrieving folder contents


Processing file 1bX4ruh0zjWcrjVvBOsWCuYVcbSeQafDb CV_1.pdf
Processing file 1WanmJg3hWAqLWOfplLPSeCSdlY6G6ZZC CV_2.pdf
Processing file 1jnOQSjl4OHKM-El8ejxu2p2Dbx1NUN6a CV_3.pdf
Processing file 1rzK1daZrsyWhFAFdTDArt_ngv_Q03-2W CV_4.pdf
Processing file 1PwUNy7t297R-bUh725V3JrE4g35OZnpI CV_5.pdf


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1bX4ruh0zjWcrjVvBOsWCuYVcbSeQafDb
To: /content/downloaded_cvs/CV_1.pdf
100%|██████████| 147k/147k [00:00<00:00, 48.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1WanmJg3hWAqLWOfplLPSeCSdlY6G6ZZC
To: /content/downloaded_cvs/CV_2.pdf
100%|██████████| 75.1k/75.1k [00:00<00:00, 61.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1jnOQSjl4OHKM-El8ejxu2p2Dbx1NUN6a
To: /content/downloaded_cvs/CV_3.pdf
100%|██████████| 72.0k/72.0k [00:00<00:00, 68.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rzK1daZrsyWhFAFdTDArt_ngv_Q03-2W
To: /content/downloaded_cvs/CV_4.pdf
100%|██████████| 73.3k/73.3k [00:00<00:00, 65.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PwUNy7t297R-bUh725V3JrE4g35OZnpI
To: /content/downloaded_cvs/CV_5.pdf
100%|██████████| 97.9k/97.9k [00:00<00:00, 17.2MB/s]
Download complete

['downloaded_cvs/CV_1.pdf',
 'downloaded_cvs/CV_2.pdf',
 'downloaded_cvs/CV_3.pdf',
 'downloaded_cvs/CV_4.pdf',
 'downloaded_cvs/CV_5.pdf']

In [4]:
# =====================================================
#  Load and display all CV PDFs in order
# =====================================================
import os
from markitdown import MarkItDown

cv_dir = "downloaded_cvs"

# Initialize MarkItDown
md = MarkItDown(enable_plugins=False)

# Collect and sort PDFs numerically
pdf_files = sorted(
    [f for f in os.listdir(cv_dir) if f.lower().endswith(".pdf")],
    key=lambda x: int("".join(filter(str.isdigit, x)))  # CV_1.pdf → 1
)

all_cvs = []

for pdf_name in pdf_files:
    pdf_path = os.path.join(cv_dir, pdf_name)
    result = md.convert(pdf_path)

    all_cvs.append({
        "file": pdf_name,
        "text": result.text_content
    })

    print("=" * 80)
    print(f"📄 {pdf_name}")
    print("=" * 80)
    print(result.text_content)
    print("\n\n")


📄 CV_1.pdf
John Smith

Marketing Professional

+ Singapore, Singapore (cid:209) Kowloon

Experience

Engineer, ByteDance

2020 – Present

• Worked in a fast-paced, global technology environment.

• Collaborated across teams to support large-scale platforms.

• Applied analytical and problem-solving skills in production systems.

Education

McGill University

Bachelor of Science (BSc) in Marketing

Skills

Content Creation

SEO

Social Media

Graduated 2009

1





📄 CV_2.pdf
Minh Pham
Design Professional

Beijing, China | Hong Kong

Professional Experience

Manager, BCG

2022 – Present

• Led cross-functional teams on client-facing design initiatives.

• Managed project timelines, deliverables, and stakeholder communication.

• Applied design thinking to business and strategy problems.

Analyst, Tencent

2013 – 2017

• Conducted market and product analysis to support decision-making.

• Collaborated with design and engineering teams.

• Produced reports and insights for senior leadersh

# Connect to our MCP server

Documentation about MCP: https://modelcontextprotocol.io/docs/getting-started/intro.

Using MCP servers in Langchain https://docs.langchain.com/oss/python/langchain/mcp.

## Check which tools that the MCP server provide

In [5]:
import asyncio
import json
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
for tool in mcp_tools:
    print(tool.name)
    print(tool.description)
    print(tool.args)
    print("\n\n------------------------------------------------------\n\n")

search_facebook_users
Search for Facebook users by display name (supports partial and fuzzy matching).

Args:
    q: Search query string (case-insensitive, matches any part of display name)
       Examples: "John", "john smith", "Smith"
    limit: Maximum number of results to return (default: 20, max: 20)
    fuzzy: Enable fuzzy matching if exact search returns no results (default: True)

Returns:
    List of user dictionaries, each containing:
    - id (int): Unique Facebook user ID for use with get_facebook_profile()
    - display_name (str): User's Facebook display name (may differ from legal name)
    - city (str): Current city of residence
    - country (str): Country of residence
    - match_type (str): "exact" or "fuzzy" (indicates search method used)
    
    Returns empty list [] if no matches found.

Example:
    search_facebook_users("Alex Chan", limit=5)
    → [{"id": 123, "display_name": "Alex Chan", "city": "Hong Kong", "country": "Hong Kong", "match_type": "exact"}]
    

## A simple agent using tools from the MCP server


In [6]:
import os
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mcp_adapters.client import MultiServerMCPClient

# ---------------------------
# 1. Define a local tool
# ---------------------------
@tool
def say_hello(name: str) -> str:
    """Say hello to a person by name."""
    return f"Hello, {name}! 👋"

# ---------------------------
# 2. Load MCP tools + merge
# ---------------------------
client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
tools = mcp_tools + [say_hello]

# ---------------------------
# 3. Initialize Gemini (tool-enabled) or deepseek
# ---------------------------
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=GEMINI_VERTEX_API_KEY,
    temperature=0,
)

# from langchain_openai import ChatOpenAI
# DEEPSEEK_API_KEY = userdata.get("DEEPSEEK_API_KEY")
# llm = ChatOpenAI(
#     model="deepseek-chat",          # or "deepseek-reasoner"
#     api_key=DEEPSEEK_API_KEY,
#     base_url="https://api.deepseek.com/v1",
#     temperature=0,
# )

llm_with_tools = llm.bind_tools(tools)

# ---------------------------
# 4. Single-step invocation
# ---------------------------
query = "Say hello to Bao using tool, then search for someone named Alice on Facebook."

response = llm_with_tools.invoke([
    HumanMessage(content=query)
])

print(response)

content='' additional_kwargs={'function_call': {'name': 'search_facebook_users', 'arguments': '{"q": "Alice"}'}} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019c6ab6-63a1-72b2-96bb-0d04926f0189-0' tool_calls=[{'name': 'say_hello', 'args': {'name': 'Bao'}, 'id': 'e71c8048-23ea-4490-b36a-28ad11b7e821', 'type': 'tool_call'}, {'name': 'search_facebook_users', 'args': {'q': 'Alice'}, 'id': '2099150b-8505-4c73-aea1-4b08f0a50a44', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 2568, 'output_tokens': 13, 'total_tokens': 2581, 'input_token_details': {'cache_read': 0}}


In [7]:
import asyncio
import json
# from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field


# ==========================================
# 0. 准备工作：获取工具引用
# ==========================================
# 从 mcp_tools 列表中按名字提取工具，方便后续调用
def get_tool_by_name(name):
    for tool in mcp_tools:
        if tool.name == name:
            return tool
    raise ValueError(f"Tool {name} not found in MCP server.")

search_linkedin = get_tool_by_name("search_linkedin_people")
get_linkedin = get_tool_by_name("get_linkedin_profile")

# ==========================================
# 1. 第一步：结构化提取 (Extractor)
# ==========================================
# 定义我们需要从 CV 里提取什么信息
class CVData(BaseModel):
    full_name: str = Field(description="Candidate's full name")
    location: str = Field(description="City or Country mentioned in CV, e.g., 'Hong Kong'")
    industry: str = Field(description="Industry keyword, e.g., 'Finance', 'Software', 'Marketing'")
    education_summary: str = Field(description="Summary of university and degree")
    experience_summary: str = Field(description="Summary of recent companies and titles")

# 定义提取链
extract_prompt = ChatPromptTemplate.from_template(
    """
    Identify the following key details from the CV text.
    If a field is not found, return an empty string.

    CV TEXT:
    {cv_text}
    """
)
# 使用 Gemini 2.0 Flash 的结构化输出能力
extract_chain = extract_prompt | llm.with_structured_output(CVData)

# ==========================================
# 2. 第三步：验证与打分 (Verifier)
# ==========================================
# 定义验证结果结构，包含一个 0-1 的分数
class VerificationResult(BaseModel):
    reasoning: str = Field(description="Explanation of discrepancies or matches")
    score: float = Field(description="Reliability score between 0.0 (Fake) and 1.0 (Valid)")

verify_prompt = ChatPromptTemplate.from_template(
    """
    You are a background check expert. Compare the CV claims against the Verified LinkedIn Profile.

    [CV DATA]
    Name: {cv_name}
    Experience: {cv_exp}
    Education: {cv_edu}

    [LINKEDIN PROFILE]
    {profile_json}

    [TASK]
    1. Check if the person exists (if profile is empty, score is 0.1).
    2. Verify key claims: Do the companies and titles match? Does the education match?
    3. Allow for minor date differences or formatting differences.
    4. Penalize HEAVILY for: Fake company names, inflated job titles (e.g., Intern -> Director), or fake degrees.

    Return a score:
    - 1.0: Perfect match
    - 0.8-0.9: Minor inconsistencies (e.g. dates off by a month)
    - 0.1-0.4: Major lies or profile not found
    """
)
verify_chain = verify_prompt | llm.with_structured_output(VerificationResult)

# ==========================================
# 3. 核心流水线逻辑 (Pipeline Controller)
# ==========================================
async def verify_single_cv(cv_text, file_name):
    print(f"\n--- Processing {file_name} ---")

    # --- Step A: Extract ---
    try:
        cv_data = await extract_chain.ainvoke({"cv_text": cv_text})
        print(f"Extracted: {cv_data.full_name} | Loc: {cv_data.location} | Ind: {cv_data.industry}")
    except Exception as e:
        print(f"Extraction failed: {e}")
        return 0.5 # 提取失败给个中间分

    # --- Step B: Search (Tool Usage) ---
    # 使用提取出的 Location 和 Industry 进行过滤，这是作业 Source 提示的技巧
    print("Searching LinkedIn...")
    try:
        search_res = await search_linkedin.ainvoke({
            "q": cv_data.full_name,
            "location": cv_data.location,
            "limit": 3,
            "fuzzy": True # Source 建议开启模糊匹配
        })
    except Exception as e:
        print(f"Search tool error: {e}")
        return 0.5

    print(search_res)


    candidates = []

    # 1. 解析工具返回的数据结构
    if isinstance(search_res, list) and len(search_res) > 0:
        try:
            # 提取 text 字段里的 JSON 字符串并解析
            candidates = json.loads(search_res[0]['text'])
        except json.JSONDecodeError:
            print(f"Error parsing JSON from tool output: {search_res['text']}")
            candidates = []



    # 2. 筛选最佳候选人 ID
    target_id = None
    target_name = None

    if candidates and isinstance(candidates, list) and len(candidates) > 0:
        print(f"Found {len(candidates)} candidates. Selecting best match...")

        # 策略：优先找 match_type 为 'exact' 的
        for cand in candidates:
            if isinstance(cand, dict) and cand.get('match_type') == 'exact':
                target_id = cand.get('id')
                target_name = cand.get('name')
                break

        # 策略：如果没找到 exact，就默认取第一个
        if target_id is None and isinstance(candidates, dict):
            target_id = candidates.get('id')
            target_name = candidates.get('name')

        print(f"Selected Profile ID: {target_id} (Name: {target_name})")
    else:
        print(f"No profile found. Raw response: {search_res}")
        return 0.1 # 找不到人，给低分



    # --- Step C: Get Profile ---
    print("Fetching full profile...")
    try:
        profile_data = await get_linkedin.ainvoke({"person_id": target_id})
    except Exception as e:
        print(f"Profile tool error: {e}")
        return 0.5

    # --- Step D: Verify & Score ---
    print("Verifying...")
    result = await verify_chain.ainvoke({
        "cv_name": cv_data.full_name,
        "cv_exp": cv_data.experience_summary,
        "cv_edu": cv_data.education_summary,
        "profile_json": str(profile_data)
    })

    print(f"Reasoning: {result.reasoning}")
    print(f"Score: {result.score}")
    return result.score


# ==========================================
# 4. 批量执行 (Batch Execution)
# ==========================================
# 这里的 all_cvs 来自你上方 Starter Code 中 MarkItDown 读取的结果
scores = []

# 依次处理每一份 CV
for cv_item in all_cvs:
    score = await verify_single_cv(cv_item['text'], cv_item['file'])
    scores.append(score)

print("\n" + "="*30)
print(f"Final Scores: {scores}")
print("="*30)



# ==========================================
# 5. 最终输出综合结果 (Results)
# ==========================================
def evaluate(scores, groundtruth, threshold=0.5):
    """
    scores: list of floats in [0, 1], length = 5
    groundtruth: list of ints (0 or 1), length = 5
    """
    assert len(scores) == 5
    assert len(groundtruth) == 5

    correct = 0
    decisions = []

    for s, gt in zip(scores, groundtruth):
        pred = 1 if s > threshold else 0
        decisions.append(pred)
        if pred == gt:
            correct += 1

    final_score = correct / len(scores)

    return {
        "decisions": decisions,
        "correct": correct,
        "total": len(scores),
        "final_score": final_score
    }

# scores = ... # Your code should generate this list [0.2, 0.3, 0.4, 0.5, 0.6]
groundtruth = [1, 1, 1, 0, 0] # Do not modify
result = evaluate(scores, groundtruth)
print(f"Final result: {result}")



--- Processing CV_1.pdf ---
Extracted: John Smith | Loc: Singapore | Ind: Marketing
Searching LinkedIn...
[{'type': 'text', 'text': '[{"id":9,"name":"John Smith","headline":"Marketing Professional","industry":"Marketing","location":"Singapore, Singapore","years_experience":7,"match_type":"exact"},{"id":31,"name":"John Smith","headline":"Logistics Professional","industry":"Logistics","location":"Singapore, Singapore","years_experience":18,"match_type":"exact"},{"id":203,"name":"John Smith","headline":"Legal Professional","industry":"Legal","location":"Singapore, Singapore","years_experience":15,"match_type":"exact"}]', 'id': 'lc_c842f7ee-2a8b-49a5-a417-fa0fdf7eace9'}]
Found 3 candidates. Selecting best match...
Selected Profile ID: 9 (Name: John Smith)
Fetching full profile...
Verifying...
Reasoning: The CV and LinkedIn profile match almost perfectly. The name, company (ByteDance), job title (Engineer), and education (McGill University, BSc in Marketing, graduation year 2009) all align